In [15]:
#Connect to Opticstudio
import clr, os, winreg
from itertools import islice

#READ ME
#For the following to function, you MUST have your optics file open in Optic Studio, and
#you MUST go to "Programming" and then click on "Interactive Extension", then run the
#python program.

# determine the Zemax working directory
aKey = winreg.OpenKey(winreg.ConnectRegistry(None, winreg.HKEY_CURRENT_USER), r"Software\Zemax", 0, winreg.KEY_READ)
zemaxData = winreg.QueryValueEx(aKey, 'ZemaxRoot')
NetHelper = os.path.join(os.sep, zemaxData[0], r'ZOS-API\Libraries\ZOSAPI_NetHelper.dll')
winreg.CloseKey(aKey)

# add the NetHelper DLL for locating the OpticStudio install folder
clr.AddReference(NetHelper)
import ZOSAPI_NetHelper

pathToInstall = ''
# uncomment the following line to use a specific instance of the ZOS-API assemblies
#pathToInstall = r'C:\C:\Program Files\Zemax OpticStudio'

# connect to OpticStudio
success = ZOSAPI_NetHelper.ZOSAPI_Initializer.Initialize(pathToInstall)

zemaxDir = ''
if success:
    zemaxDir = ZOSAPI_NetHelper.ZOSAPI_Initializer.GetZemaxDirectory()
    print('Found OpticStudio at:   %s' + zemaxDir)
else:
    raise Exception('Cannot find OpticStudio')

# load the ZOS-API assemblies
clr.AddReference(os.path.join(os.sep, zemaxDir, r'ZOSAPI.dll'))
clr.AddReference(os.path.join(os.sep, zemaxDir, r'ZOSAPI_Interfaces.dll'))
import ZOSAPI

TheConnection = ZOSAPI.ZOSAPI_Connection()
if TheConnection is None:
    raise Exception("Unable to intialize NET connection to ZOSAPI")

TheApplication = TheConnection.ConnectAsExtension(0)
if TheApplication is None:
    raise Exception("Unable to acquire ZOSAPI application")

if TheApplication.IsValidLicenseForAPI == False:
    raise Exception("License is not valid for ZOSAPI use.  Make sure you have enabled 'Programming > Interactive Extension' from the OpticStudio GUI.")

TheSystem = TheApplication.PrimarySystem
if TheSystem is None:
    raise Exception("Unable to acquire Primary system")

print('Connected to OpticStudio')
####################################################################################################

#Program Setup
import numpy as np
import scipy
import scipy.io as scipyio
import sys, time
import matplotlib.pyplot as plt
from astropy.io import fits
import csv

def transpose(data):
    """Transposes a 2D list (Python3.x or greater).  
    
    Useful for converting mutli-dimensional line series (i.e. FFT PSF)
    
    Parameters
    ----------
    data      : Python native list (if using System.Data[,] object reshape first)    
    
    Returns
    -------
    res       : transposed 2D list
    """
    if type(data) is not list:
        data = list(data)
    return list(map(list, zip(*data)))
    
def reshape(data, x, y, transpose = False):
    """Converts a System.Double[,] to a 2D list for plotting or post processing
    
    Parameters
    ----------
    data      : System.Double[,] data directly from ZOS-API 
    x         : x width of new 2D list [use var.GetLength(0) for dimension]
    y         : y width of new 2D list [use var.GetLength(1) for dimension]
    transpose : transposes data; needed for some multi-dimensional line series data
    
    Returns
    -------
    res       : 2D list; can be directly used with Matplotlib or converted to
                a numpy array using numpy.asarray(res)
    """
    if type(data) is not list:
        data = list(data)
    var_lst = [y] * x;
    it = iter(data)
    res = [list(islice(it, i)) for i in var_lst]
    if transpose:
        return self.transpose(res);
    return res

##STAR Setup
analysis = TheSystem.Analyses
tools = TheSystem.Tools
surf_inds = [3,8,13,18,23,27,31,36,40,44]
surf_names = ['M1','M2','M3','LODM','Dicro','OAP1','OAP2','BMC','OAP3','VVC']
n_surf = len(surf_inds)

Found OpticStudio at:   %sc:\program files\ansys zemax opticstudio 2022 r2.02
Connected to OpticStudio


In [16]:
##Setup
data_dir = 'c:\\Users\\locsst\\Desktop\\TD_picture_working\\ansys\\STAR\\am\\' 
mf_dir = 'c:\\Users\\locsst\\Documents\\Zemax\\MeritFunction\\'
nsteps = 10

TheLDE = TheSystem.LDE
move_m2 = TheLDE.GetSurfaceAt(7)
surf_lodm = TheLDE.GetSurfaceAt(18)
surf_vvc = TheLDE.GetSurfaceAt(44)

chief_arr = np.zeros((n_surf,2,nsteps))
nominal_chief_arr = np.zeros((n_surf,2))
zern_amp = np.zeros((nsteps,12))

wf = TheSystem.Analyses.New_Analysis(ZOSAPI.Analysis.AnalysisIDM.WavefrontMap)
wfsettings = wf.GetSettings()
wfsettings.Sampling = ZOSAPI.Analysis.SampleSizes.S_128x128
wfsettings.ShowAs = ZOSAPI.Analysis.Settings.Wavefront.FoucaultShowAs.FalseColor

wf.ApplyAndWaitForCompletion()
wfresults = wf.GetResults()
wfgrid = wfresults.GetDataGrid(0).Values
wfsize = [wfgrid.GetLength(0),wfgrid.GetLength(1)]

wfcube = np.zeros((wfsize[0],wfsize[1],nsteps))

In [17]:
##Clear Any Leftover STAR Data from previous runs, reset variables for M2 correction
for i in range(n_surf):
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.UnloadData()

move_m2.ThicknessCell.set_DoubleValue(0)
move_m2.GetCellAt(12).set_DoubleValue(0)
move_m2.GetCellAt(13).set_DoubleValue(0)
move_m2.GetCellAt(14).set_DoubleValue(0)
move_m2.GetCellAt(15).set_DoubleValue(0)

TheLDE.GetSurfaceAt(17).GetCellAt(14).set_DoubleValue(0)
TheLDE.GetSurfaceAt(17).GetCellAt(15).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par18).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par19).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par20).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par21).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par22).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par23).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par24).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par25).set_DoubleValue(0)
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par26).set_DoubleValue(0)

#Load OPD Merit Function
TheSystem.MFE.LoadMeritFunction(mf_dir + 'opd.mf')

#Get data from nominal system
for i in range(n_surf):
    nominal_chief_arr[i,0] = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.REAX, surf_inds[i],0,0,0,0,0,0,0)
    nominal_chief_arr[i,1] = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.REAY, surf_inds[i],0,0,0,0,0,0,0)

vvc_x0 = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGX, 44,0,0,0,0,0,0,0)
vvc_y0 = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGY, 44,0,0,0,0,0,0,0)

wf.ApplyAndWaitForCompletion()

In [4]:
print(vvc_x0,vvc_y0)

-8.460908727873708 1.5469406989124917e-11


In [18]:
##Run initial timestep
for i in range(n_surf):
    disp_file = data_dir + '00_Timestep_00001_20240220' + '\\Surface_' + str(surf_inds[i]).zfill(2) + '_Deformation.txt'
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.SetDataIsGlobal()
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.Fits.Settings.SetAllValues(2,2,2,10,10**(-9),0.5)
    
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.ImportDeformations(disp_file)
    time.sleep(0.2)
    
wf.ApplyAndWaitForCompletion()

In [20]:
##Apply Secondary Correction
tools.RemoveAllVariables()

move_m2.ThicknessCell.MakeSolveVariable()
move_m2.GetCellAt(12).MakeSolveVariable()
move_m2.GetCellAt(13).MakeSolveVariable()
move_m2.GetCellAt(14).MakeSolveVariable()
move_m2.GetCellAt(15).MakeSolveVariable()

surf_vvc.STARData.Deformations.RBMs.GetTransformValues()
vvc_dx = surf_vvc.STARData.Deformations.RBMs.Decenters.X
vvc_dy = surf_vvc.STARData.Deformations.RBMs.Decenters.Y

TheSystem.MFE.GetOperandAt(4).set_Target(vvc_x0 + vvc_dx)
TheSystem.MFE.GetOperandAt(5).set_Target(vvc_y0 + vvc_dy)

LocalOpt = TheSystem.Tools.OpenLocalOptimization()
if (LocalOpt != None):

    LocalOpt.Algorithm = ZOSAPI.Tools.Optimization.OptimizationAlgorithm.OrthogonalDescent
    LocalOpt.Cycles = ZOSAPI.Tools.Optimization.OptimizationCycles.Fixed_50_Cycles
    LocalOpt.NumberOfCores = 32
    print('Initial Merit Function ', LocalOpt.InitialMeritFunction)
    
    LocalOpt.RunAndWaitWithTimeout(60)
    LocalOpt.Cancel()
    LocalOpt.WaitForCompletion()
    print('Intermediate Merit Function: ', LocalOpt.CurrentMeritFunction)

    LocalOpt.Algorithm = ZOSAPI.Tools.Optimization.OptimizationAlgorithm.DampedLeastSquares
    LocalOpt.Cycles = ZOSAPI.Tools.Optimization.OptimizationCycles.Infinite
    LocalOpt.RunAndWaitWithTimeout(60)
    LocalOpt.Cancel()
    LocalOpt.WaitForCompletion()
    print('Final Merit Function: ', LocalOpt.CurrentMeritFunction)
    
    LocalOpt.Close()

print('Deviation in X (um): ', 2.54*(10**4)*(vvc_x0 + vvc_dx-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGX, 44,0,0,0,0,0,0,0)))
print('Deviation in Y (um): ', 2.54*(10**4)*(vvc_y0 + vvc_dy-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGY, 44,0,0,0,0,0,0,0)))

wf.ApplyAndWaitForCompletion()

Initial Merit Function  0.0018421249437018674
Intermediate Merit Function:  0.001816319714464698
Final Merit Function:  0.0018162991727661622
Deviation in X (um):  -0.4694137253725472
Deviation in Y (um):  0.9004956876948744


In [21]:
print(vvc_dx,vvc_dy)

-0.012676687563846779 0.018635894526067873


In [22]:
##Apply LODM correction to first timestep
tools.RemoveAllVariables()

TheSystem.MFE.LoadMeritFunction(mf_dir + 'zernike.mf')
time.sleep(0.5)

TheLDE.GetSurfaceAt(17).GetCellAt(14).MakeSolveVariable()
TheLDE.GetSurfaceAt(17).GetCellAt(15).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par18).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par19).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par20).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par21).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par22).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par23).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par24).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par25).MakeSolveVariable()
surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par26).MakeSolveVariable()

TheSystem.MFE.GetOperandAt(4).set_Target(vvc_x0 + vvc_dx)
TheSystem.MFE.GetOperandAt(5).set_Target(vvc_y0 + vvc_dy)

#Optimize Zernikes at LODM
LocalOpt = TheSystem.Tools.OpenLocalOptimization()
if (LocalOpt != None):
    LocalOpt.Algorithm = ZOSAPI.Tools.Optimization.OptimizationAlgorithm.DampedLeastSquares
    LocalOpt.Cycles = ZOSAPI.Tools.Optimization.OptimizationCycles.Infinite
    LocalOpt.NumberOfCores = 32
    print('Initial Merit Function ', LocalOpt.InitialMeritFunction)
    
    LocalOpt.RunAndWaitWithTimeout(45)
    LocalOpt.Cancel()
    LocalOpt.WaitForCompletion()
    print('Final Merit Function: ', LocalOpt.CurrentMeritFunction)
    LocalOpt.Close()

print('Deviation in X (um): ', 2.54*(10**4)*(vvc_x0 + vvc_dx-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGX, 44,0,0,0,0,0,0,0)))
print('Deviation in Y (um): ', 2.54*(10**4)*(vvc_y0 + vvc_dy-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGY, 44,0,0,0,0,0,0,0)))

#Record Zernike Amplitudes
zern_amp[0,1] = TheLDE.GetSurfaceAt(17).GetCellAt(14).get_Value()
zern_amp[0,2] = TheLDE.GetSurfaceAt(17).GetCellAt(15).get_Value()
zern_amp[0,3] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par18).get_Value()
zern_amp[0,4] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par19).get_Value()
zern_amp[0,5] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par20).get_Value()
zern_amp[0,6] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par21).get_Value()
zern_amp[0,7] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par22).get_Value()
zern_amp[0,8] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par23).get_Value()
zern_amp[0,9] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par24).get_Value()
zern_amp[0,10] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par25).get_Value()
zern_amp[0,11] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par26).get_Value()

#Get chief ray location at each optic (in lens coords)
for i in range(n_surf):
    chief_arr[i,0,0] = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.REAX, surf_inds[i],0,0,0,0,0,0,0)
    chief_arr[i,1,0] = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.REAY, surf_inds[i],0,0,0,0,0,0,0)

    #Must include motion of the optic!
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.RBMs.GetTransformValues()
    chief_arr[i,0,0] += TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.RBMs.Decenters.X
    chief_arr[i,1,0] += TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.RBMs.Decenters.Y

wf.ApplyAndWaitForCompletion()
wfresults = wf.GetResults()
wfgrid = wfresults.GetDataGrid(0).Values
wfarr = reshape(wfgrid,wfgrid.GetLength(0),wfgrid.GetLength(1))

wfcube[:,:,0] = wfarr

Initial Merit Function  0.005623915834598284
Final Merit Function:  2.4617790605248974e-05
Deviation in X (um):  -0.02853895958310204
Deviation in Y (um):  0.8400375064227319


In [24]:
#Clear STAR Data for start of looping
for i in range(n_surf):
    TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.UnloadData()

for j in range(1,nsteps):
    print(f'Timestep: {str(j)} of {str(nsteps-1)}')
    
    #Open displacement data
    for i in range(n_surf):
        disp_file = data_dir + str(j).zfill(2) + '_Timestep_0'+str(j).zfill(2)+'00_20240220' + '\\Surface_' + str(surf_inds[i]).zfill(2) + '_Deformation.txt'
        TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.ImportDeformations(disp_file)

        time.sleep(0.2)
    
    wf.ApplyAndWaitForCompletion()

    surf_vvc.STARData.Deformations.RBMs.GetTransformValues()
    vvc_dx = surf_vvc.STARData.Deformations.RBMs.Decenters.X
    vvc_dy = surf_vvc.STARData.Deformations.RBMs.Decenters.Y

    TheSystem.MFE.GetOperandAt(4).set_Target(vvc_x0 + vvc_dx)
    TheSystem.MFE.GetOperandAt(5).set_Target(vvc_y0 + vvc_dy)

    #Optimize Zernikes at LODM
    print('Looping: 0',end='\n')
    LocalOpt = TheSystem.Tools.OpenLocalOptimization()
    if (LocalOpt != None):
        LocalOpt.Algorithm = ZOSAPI.Tools.Optimization.OptimizationAlgorithm.DampedLeastSquares
        LocalOpt.Cycles = ZOSAPI.Tools.Optimization.OptimizationCycles.Infinite
        LocalOpt.NumberOfCores = 32
        print('Initial Merit Function ', LocalOpt.InitialMeritFunction)
        
        LocalOpt.RunAndWaitWithTimeout(30)
        LocalOpt.Cancel()
        LocalOpt.WaitForCompletion()
        LocalOpt.Close()

    xdev = 2.54*(10**4)*(vvc_x0 + vvc_dx-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGX, 44,0,0,0,0,0,0,0))
    ydev = 2.54*(10**4)*(vvc_y0 + vvc_dy-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGY, 44,0,0,0,0,0,0,0))

    loopct = 0
    while np.sqrt(xdev**2 + ydev**2) > 1.0:
        loopct += 1
        print('Looping: ', loopct,end='\n')
        LocalOpt = TheSystem.Tools.OpenLocalOptimization()
        if (LocalOpt != None):
            LocalOpt.RunAndWaitWithTimeout(30)
            LocalOpt.Cancel()
            LocalOpt.WaitForCompletion()
            merit = LocalOpt.CurrentMeritFunction
            LocalOpt.Close()
        
        xdev = 2.54*(10**4)*(vvc_x0 + vvc_dx-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGX, 44,0,0,0,0,0,0,0))
        ydev = 2.54*(10**4)*(vvc_y0 + vvc_dy-TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.RAGY, 44,0,0,0,0,0,0,0))

        if loopct >= 5:
            break

    print('')
    print('Final Merit Function: ', merit)
    print('Deviation in X (um): ', xdev)
    print('Deviation in Y (um): ', ydev)

    #Record Zernike Amplitudes
    zern_amp[j,1] = TheLDE.GetSurfaceAt(17).GetCellAt(14).get_Value()
    zern_amp[j,2] = TheLDE.GetSurfaceAt(17).GetCellAt(15).get_Value()
    zern_amp[j,3] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par18).get_Value()
    zern_amp[j,4] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par19).get_Value()
    zern_amp[j,5] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par20).get_Value()
    zern_amp[j,6] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par21).get_Value()
    zern_amp[j,7] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par22).get_Value()
    zern_amp[j,8] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par23).get_Value()
    zern_amp[j,9] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par24).get_Value()
    zern_amp[j,10] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par25).get_Value()
    zern_amp[j,11] = surf_lodm.GetSurfaceCell(ZOSAPI.Editors.LDE.SurfaceColumn.Par26).get_Value()

    #Get chief ray location at each optic (in lens coords)
    for i in range(n_surf):
        chief_arr[i,0,j] = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.REAX, surf_inds[i],0,0,0,0,0,0,0)
        chief_arr[i,1,j] = TheSystem.MFE.GetOperandValue(ZOSAPI.Editors.MFE.MeritOperandType.REAY, surf_inds[i],0,0,0,0,0,0,0)

        #Must include motion of the optic!
        TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.RBMs.GetTransformValues()
        chief_arr[i,0,j] += TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.RBMs.Decenters.X
        chief_arr[i,1,j] += TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.RBMs.Decenters.Y

    wf.ApplyAndWaitForCompletion()
    wfresults = wf.GetResults()
    wfgrid = wfresults.GetDataGrid(0).Values
    wfarr = reshape(wfgrid,wfgrid.GetLength(0),wfgrid.GetLength(1))

    wfcube[:,:,j] = wfarr

    #Clear STAR Data for next timestep
    for i in range(n_surf):
        TheLDE.GetSurfaceAt(surf_inds[i]).STARData.Deformations.FEAData.UnloadData()

Timestep: 1 of 9
Initial Merit Function  0.006051896927368291
Looping:  1Looping:  2Looping:  3Looping:  4Looping:  5
Final Merit Function:  5.603751286082933e-05
Deviation in X (um):  -0.16609879800704164
Deviation in Y (um):  1.994479962310626
Timestep: 2 of 9
Initial Merit Function  0.006705187338861622

Final Merit Function:  5.603751286082933e-05
Deviation in X (um):  0.1168924224216994
Deviation in Y (um):  0.1393934024960311
Timestep: 3 of 9
Initial Merit Function  0.006151594580154354
Looping:  1Looping:  2Looping:  3Looping:  4Looping:  5
Final Merit Function:  3.2308866737055896e-05
Deviation in X (um):  0.08326066789194897
Deviation in Y (um):  1.1441390231249202
Timestep: 4 of 9
Initial Merit Function  0.005615352010436906


In [ ]:
bwalk = np.zeros((n_surf,2,nsteps))

for k in range(nsteps):
    bwalk[:,:,k] = chief_arr[:,:,k] - nominal_chief_arr

axis = ['X','Y']

tarr = np.zeros(nsteps)
for i in range(nsteps):
    tarr[i] = 5./3 * i

for j in range(2):
    minv = np.min(bwalk[:,j,:])
    maxv = np.max(bwalk[:,j,:])

    minv -= 0.1*(maxv-minv)
    maxv += 0.1*(maxv-minv)

    plt.figure()
    plt.ylim(minv,maxv)
    plt.title(f'AM Beamwalk over time : {axis[j]} Direction')
    plt.xlabel('Time (Mins)')
    plt.ylabel('Beamwalk (in)')

    for i in range(n_surf):
        plt.plot(tarr,bwalk[i,j,:])

    plt.legend(surf_names,bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
bwalk = np.zeros((n_surf,2,nsteps))

for k in range(nsteps):
    bwalk[:,:,k] = chief_arr[:,:,k] - nominal_chief_arr

axis = ['X','Y']

tarr = np.zeros(nsteps)
for i in range(nsteps):
    tarr[i] = 5./3 * i

for j in range(2):
    minv = np.min(bwalk[:,j,:])
    maxv = np.max(bwalk[:,j,:])

    minv -= 0.1*(maxv-minv)
    maxv += 0.1*(maxv-minv)

    plt.figure()
    plt.ylim(minv,maxv)
    plt.title(f'PM Beamwalk over time : {axis[j]} Direction')
    plt.xlabel('Time (Mins)')
    plt.ylabel('Beamwalk (in)')

    for i in range(n_surf):
        plt.plot(tarr,bwalk[i,j,:])

    plt.legend(surf_names,bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

In [ ]:
##Save Data
outdir = 'C:\\Users\\locsst\\Desktop\\TD_picture_working\\ansys\\STAR\\STAR_res\\'

#Beamwalk 
filename = 'am_beamwalk.csv'

#Create output array
big_arr = np.zeros((n_surf,2*nsteps+1))
big_arr[:,0] = [1,3,5,7,16,17,19,21,25]
for i in range(nsteps):
    big_arr[:,2*i+1] = 0.0254*bwalk[:,0,i]
    big_arr[:,2*i+2]   = 0.0254*bwalk[:,1,i]


with open(outdir+filename,'w',newline = '') as file:
    writen = csv.writer(file)
    writen.writerows(big_arr)

#Wavefront

#Calculate Sampling
r0 = 11.7064 #Primary Clear Radius
pixsize = r0/128 * 0.0254

#Create Header
header = fits.Header()
header['SCALE'] = str(pixsize)
header['UNITS'] = 'meters'

#Convert Data to Meters and write to file
for i in range(nsteps):
    filename = 'am'+str(i+1) + '.fits'
    primary = fits.PrimaryHDU((6 *10**(-7))*wfcube[:,:,i],header=header)
    primary.writeto(outdir + filename, overwrite='True')
